In [1]:
import qlib
from qlib.constant import REG_CN
from qlib.data import D
import pandas as pd
provider_uri = "C:/Users/huangtuo/.qlib/qlib_data/cn_data/"  # target_dir
qlib.init(provider_uri=provider_uri, region=REG_CN)
from datetime import datetime

test_period = ("2005-01-01", "2024-05-15")

[23392:MainThread](2024-07-04 10:06:31,087) INFO - qlib.Initialization - [config.py:416] - default_conf: client.
[23392:MainThread](2024-07-04 10:06:31,872) INFO - qlib.Initialization - [__init__.py:74] - qlib successfully initialized based on client settings.
[23392:MainThread](2024-07-04 10:06:31,873) INFO - qlib.Initialization - [__init__.py:76] - data_path={'__DEFAULT_FREQ': WindowsPath('C:/Users/huangtuo/.qlib/qlib_data/cn_data')}


In [2]:
market = "csi300"
#benchmark = "SZ16070"
benchmark = "SH000300"


In [3]:
from qlib.data import D
import talib
from typing import List, Tuple, Dict

In [4]:
# 获取test时段的行情原始数据
stockpool: List = D.instruments(market=market)
raw_data: pd.DataFrame = D.features(
    stockpool,
    fields=["$open", "$high", "$low", "$close", "$volume"],
    start_time=test_period[0],
    end_time=test_period[1],
)

In [5]:
raw_data.head()

$open     $high      $low    $close       $volume
instrument datetime                                                        
SH600000   2005-04-08  1.075838  1.102007  1.064207  1.093284  683986.37500
           2005-04-11  1.094433  1.120595  1.078445  1.088619  943351.56250
           2005-04-12  1.087297  1.090204  1.069854  1.072761  446335.87500
           2005-04-13  1.075131  1.117264  1.075131  1.086754  644413.81250
           2005-04-14  1.086989  1.095708  1.065191  1.068097  382452.84375

In [6]:
raw_data.loc[[( 'SH600000')], :]

$open      $high       $low     $close  \
instrument datetime                                                 
SH600000   2005-04-08   1.075838   1.102007   1.064207   1.093284   
           2005-04-11   1.094433   1.120595   1.078445   1.088619   
           2005-04-12   1.087297   1.090204   1.069854   1.072761   
           2005-04-13   1.075131   1.117264   1.075131   1.086754   
           2005-04-14   1.086989   1.095708   1.065191   1.068097   
...                          ...        ...        ...        ...   
           2024-05-08  11.318597  11.376567  11.275120  11.304105   
           2024-05-09  11.304105  11.376567  11.246135  11.304105   
           2024-05-10  11.319054  11.521955  11.304561  11.507463   
           2024-05-13  11.492902  11.565367  11.405944  11.536381   
           2024-05-14  11.551148  11.565641  11.391723  11.420709   

                            $volume  
instrument datetime                  
SH600000   2005-04-08  683986.37500  
           2005-04-11  943351.56250  
           2005-04-12  446335.87500  
           2005-04-13  644413.81250  
           2005-04-14  382452.84375  
...                             ...  
           2024-05-08  209611.96875  
           2024-05-09  234205.46875  
           2024-05-10  360253.34375  
           2024-05-13  278909.03125  
           2024-05-14  274911.03125  

[4640 rows x 5 columns]

In [7]:
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import pandas as pd

def knn_stock_prediction(df, n_neighbors=5, split_percentage=0.95):
    """
    使用 K-Nearest Neighbors (KNN) 算法进行股票价格预测。

    参数:
    df (pandas.DataFrame): 包含股票数据的 DataFrame
    n_neighbors (int): KNN 算法中的 n_neighbors 参数
    split_percentage (float): 训练集和测试集的分割比例

    返回:
    df (pandas.DataFrame): 包含预测信号的 DataFrame
    
    """
    df = df.dropna()
    #df = df[['Open', 'High', 'Low', 'Close']]
    #df['Open'] = df['Open'].astype(np.float64)
    #df['High'] = df['High'].astype(np.float64)
    #df['Low'] = df['Low'].astype(np.float64)
    #df['Close'] = df['Close'].astype(np.float64)
    
    instruments = df.index.get_level_values('instrument').unique()
    predicted_signals = []

    for instrument in instruments:
        instrument_df = df.loc[instrument]

        # 特征工程
        instrument_df['Open-Close'] = instrument_df['open'] - instrument_df['close']
        instrument_df['High-Low'] = instrument_df['high'] - instrument_df['low']
        X = instrument_df[['Open-Close', 'High-Low']]

        # 目标变量
        Y = np.where(instrument_df['close'].shift(-1) > instrument_df['close'], 1, -1)

        # 数据集拆分
        split = int(split_percentage * len(instrument_df))
        X_train = X[:split]
        Y_train = Y[:split]
        X_test = X[split:]
        Y_test = Y[split:]

        # 训练 KNN 模型
        knn = KNeighborsClassifier(n_neighbors=n_neighbors)
        knn.fit(X_train, Y_train)

        # 计算准确率
        accuracy_train = accuracy_score(Y_train, knn.predict(X_train))
        accuracy_test = accuracy_score(Y_test, knn.predict(X_test))
        print(f'Instrument: {instrument}, Train_data Accuracy: {accuracy_train:.2f}, Test_data Accuracy: {accuracy_test:.2f}')

        # 生成预测信号
        instrument_df['Predicted_Signal'] = knn.predict(X)
        #instrument_df['Predicted_Signal'] = instrument_df['Predicted_Signal'].where(instrument_df['Predicted_Signal'] != -1, 0)
        predicted_signals.append(instrument_df['Predicted_Signal'])

    # 合并预测信号
    df['Predicted_Signal'] = pd.concat(predicted_signals, keys=instruments).reorder_levels(['instrument', 'datetime'])

    return df

In [8]:
# 替换列名中的特殊字符
raw_data.columns = [col.replace('$', '') for col in raw_data.columns]
test=knn_stock_prediction(raw_data)

Instrument: SH600000, Train_data Accuracy: 0.68, Test_data Accuracy: 0.59
Instrument: SH600001, Train_data Accuracy: 0.70, Test_data Accuracy: 0.50
Instrument: SH600002, Train_data Accuracy: 0.69, Test_data Accuracy: 0.42
Instrument: SH600003, Train_data Accuracy: 0.71, Test_data Accuracy: 0.50
Instrument: SH600004, Train_data Accuracy: 0.67, Test_data Accuracy: 0.51
Instrument: SH600005, Train_data Accuracy: 0.69, Test_data Accuracy: 0.47
Instrument: SH600006, Train_data Accuracy: 0.68, Test_data Accuracy: 0.46
Instrument: SH600007, Train_data Accuracy: 0.69, Test_data Accuracy: 0.48
Instrument: SH600008, Train_data Accuracy: 0.67, Test_data Accuracy: 0.49
Instrument: SH600009, Train_data Accuracy: 0.70, Test_data Accuracy: 0.48
Instrument: SH600010, Train_data Accuracy: 0.70, Test_data Accuracy: 0.63
Instrument: SH600011, Train_data Accuracy: 0.67, Test_data Accuracy: 0.55
Instrument: SH600012, Train_data Accuracy: 0.72, Test_data Accuracy: 0.52
Instrument: SH600015, Train_data Accur

Instrument: SH600259, Train_data Accuracy: 0.72, Test_data Accuracy: 0.63
Instrument: SH600266, Train_data Accuracy: 0.69, Test_data Accuracy: 0.54
Instrument: SH600267, Train_data Accuracy: 0.69, Test_data Accuracy: 0.49
Instrument: SH600269, Train_data Accuracy: 0.68, Test_data Accuracy: 0.49
Instrument: SH600270, Train_data Accuracy: 0.66, Test_data Accuracy: 0.37
Instrument: SH600271, Train_data Accuracy: 0.68, Test_data Accuracy: 0.45
Instrument: SH600276, Train_data Accuracy: 0.69, Test_data Accuracy: 0.49
Instrument: SH600277, Train_data Accuracy: 0.65, Test_data Accuracy: 0.67
Instrument: SH600282, Train_data Accuracy: 0.71, Test_data Accuracy: 0.45
Instrument: SH600296, Train_data Accuracy: 0.74, Test_data Accuracy: 0.33
Instrument: SH600297, Train_data Accuracy: 0.69, Test_data Accuracy: 0.47
Instrument: SH600299, Train_data Accuracy: 0.69, Test_data Accuracy: 0.62
Instrument: SH600300, Train_data Accuracy: 0.67, Test_data Accuracy: 0.47
Instrument: SH600307, Train_data Accur

Instrument: SH600637, Train_data Accuracy: 0.68, Test_data Accuracy: 0.51
Instrument: SH600638, Train_data Accuracy: 0.69, Test_data Accuracy: 0.44
Instrument: SH600639, Train_data Accuracy: 0.68, Test_data Accuracy: 0.50
Instrument: SH600641, Train_data Accuracy: 0.73, Test_data Accuracy: 0.67
Instrument: SH600642, Train_data Accuracy: 0.69, Test_data Accuracy: 0.55
Instrument: SH600643, Train_data Accuracy: 0.70, Test_data Accuracy: 0.48
Instrument: SH600648, Train_data Accuracy: 0.68, Test_data Accuracy: 0.52
Instrument: SH600649, Train_data Accuracy: 0.68, Test_data Accuracy: 0.51
Instrument: SH600652, Train_data Accuracy: 0.68, Test_data Accuracy: 0.62
Instrument: SH600653, Train_data Accuracy: 0.69, Test_data Accuracy: 0.45
Instrument: SH600654, Train_data Accuracy: 0.68, Test_data Accuracy: 0.38
Instrument: SH600655, Train_data Accuracy: 0.69, Test_data Accuracy: 0.49
Instrument: SH600657, Train_data Accuracy: 0.68, Test_data Accuracy: 0.33
Instrument: SH600660, Train_data Accur

Instrument: SH600970, Train_data Accuracy: 0.71, Test_data Accuracy: 0.39
Instrument: SH600971, Train_data Accuracy: 0.67, Test_data Accuracy: 0.56
Instrument: SH600977, Train_data Accuracy: 0.72, Test_data Accuracy: 0.60
Instrument: SH600978, Train_data Accuracy: 0.66, Test_data Accuracy: 0.44
Instrument: SH600989, Train_data Accuracy: 0.70, Test_data Accuracy: 0.47
Instrument: SH600997, Train_data Accuracy: 0.69, Test_data Accuracy: 0.45
Instrument: SH600998, Train_data Accuracy: 0.69, Test_data Accuracy: 0.45
Instrument: SH600999, Train_data Accuracy: 0.68, Test_data Accuracy: 0.56
Instrument: SH601001, Train_data Accuracy: 0.69, Test_data Accuracy: 0.53
Instrument: SH601002, Train_data Accuracy: 0.65, Test_data Accuracy: 0.62
Instrument: SH601003, Train_data Accuracy: 0.69, Test_data Accuracy: 0.42
Instrument: SH601005, Train_data Accuracy: 0.67, Test_data Accuracy: 0.44
Instrument: SH601006, Train_data Accuracy: 0.69, Test_data Accuracy: 0.51
Instrument: SH601009, Train_data Accur

Instrument: SH601865, Train_data Accuracy: 0.70, Test_data Accuracy: 0.59
Instrument: SH601866, Train_data Accuracy: 0.70, Test_data Accuracy: 0.59
Instrument: SH601868, Train_data Accuracy: 0.72, Test_data Accuracy: 0.55
Instrument: SH601872, Train_data Accuracy: 0.69, Test_data Accuracy: 0.46
Instrument: SH601877, Train_data Accuracy: 0.67, Test_data Accuracy: 0.62
Instrument: SH601878, Train_data Accuracy: 0.68, Test_data Accuracy: 0.51
Instrument: SH601881, Train_data Accuracy: 0.70, Test_data Accuracy: 0.50
Instrument: SH601888, Train_data Accuracy: 0.67, Test_data Accuracy: 0.52
Instrument: SH601898, Train_data Accuracy: 0.70, Test_data Accuracy: 0.52
Instrument: SH601899, Train_data Accuracy: 0.69, Test_data Accuracy: 0.46
Instrument: SH601901, Train_data Accuracy: 0.69, Test_data Accuracy: 0.49
Instrument: SH601916, Train_data Accuracy: 0.69, Test_data Accuracy: 0.66
Instrument: SH601918, Train_data Accuracy: 0.69, Test_data Accuracy: 0.38
Instrument: SH601919, Train_data Accur

Instrument: SZ000157, Train_data Accuracy: 0.69, Test_data Accuracy: 0.55
Instrument: SZ000166, Train_data Accuracy: 0.70, Test_data Accuracy: 0.46
Instrument: SZ000301, Train_data Accuracy: 0.68, Test_data Accuracy: 0.49
Instrument: SZ000333, Train_data Accuracy: 0.69, Test_data Accuracy: 0.55
Instrument: SZ000338, Train_data Accuracy: 0.69, Test_data Accuracy: 0.43
Instrument: SZ000400, Train_data Accuracy: 0.71, Test_data Accuracy: 0.48
Instrument: SZ000401, Train_data Accuracy: 0.69, Test_data Accuracy: 0.57
Instrument: SZ000402, Train_data Accuracy: 0.69, Test_data Accuracy: 0.56
Instrument: SZ000406, Train_data Accuracy: 0.66, Test_data Accuracy: 0.50
Instrument: SZ000408, Train_data Accuracy: 0.69, Test_data Accuracy: 0.74
Instrument: SZ000410, Train_data Accuracy: 0.68, Test_data Accuracy: 0.44
Instrument: SZ000413, Train_data Accuracy: 0.68, Test_data Accuracy: 0.55
Instrument: SZ000415, Train_data Accuracy: 0.68, Test_data Accuracy: 0.35
Instrument: SZ000420, Train_data Accur

Instrument: SZ000823, Train_data Accuracy: 0.70, Test_data Accuracy: 0.89
Instrument: SZ000825, Train_data Accuracy: 0.70, Test_data Accuracy: 0.52
Instrument: SZ000826, Train_data Accuracy: 0.69, Test_data Accuracy: 0.62
Instrument: SZ000828, Train_data Accuracy: 0.70, Test_data Accuracy: 0.56
Instrument: SZ000829, Train_data Accuracy: 0.68, Test_data Accuracy: 0.52
Instrument: SZ000831, Train_data Accuracy: 0.67, Test_data Accuracy: 0.59
Instrument: SZ000839, Train_data Accuracy: 0.69, Test_data Accuracy: 0.45
Instrument: SZ000858, Train_data Accuracy: 0.69, Test_data Accuracy: 0.53
Instrument: SZ000860, Train_data Accuracy: 0.69, Test_data Accuracy: 0.37
Instrument: SZ000866, Train_data Accuracy: 0.67, Test_data Accuracy: 0.33
Instrument: SZ000869, Train_data Accuracy: 0.70, Test_data Accuracy: 0.49
Instrument: SZ000875, Train_data Accuracy: 0.67, Test_data Accuracy: 0.29
Instrument: SZ000876, Train_data Accuracy: 0.69, Test_data Accuracy: 0.46
Instrument: SZ000877, Train_data Accur

Instrument: SZ002310, Train_data Accuracy: 0.69, Test_data Accuracy: 0.48
Instrument: SZ002311, Train_data Accuracy: 0.70, Test_data Accuracy: 0.42
Instrument: SZ002344, Train_data Accuracy: 0.69, Test_data Accuracy: 0.64
Instrument: SZ002352, Train_data Accuracy: 0.71, Test_data Accuracy: 0.52
Instrument: SZ002353, Train_data Accuracy: 0.70, Test_data Accuracy: 0.60
Instrument: SZ002371, Train_data Accuracy: 0.70, Test_data Accuracy: 0.45
Instrument: SZ002375, Train_data Accuracy: 0.68, Test_data Accuracy: 0.46
Instrument: SZ002378, Train_data Accuracy: 0.73, Test_data Accuracy: 0.46
Instrument: SZ002384, Train_data Accuracy: 0.67, Test_data Accuracy: 0.54
Instrument: SZ002385, Train_data Accuracy: 0.69, Test_data Accuracy: 0.55
Instrument: SZ002399, Train_data Accuracy: 0.69, Test_data Accuracy: 0.40
Instrument: SZ002400, Train_data Accuracy: 0.71, Test_data Accuracy: 0.46
Instrument: SZ002405, Train_data Accuracy: 0.70, Test_data Accuracy: 0.50
Instrument: SZ002410, Train_data Accur

Instrument: SZ300454, Train_data Accuracy: 0.73, Test_data Accuracy: 0.30
Instrument: SZ300496, Train_data Accuracy: 0.69, Test_data Accuracy: 0.52
Instrument: SZ300498, Train_data Accuracy: 0.69, Test_data Accuracy: 0.55
Instrument: SZ300529, Train_data Accuracy: 0.68, Test_data Accuracy: 0.55
Instrument: SZ300558, Train_data Accuracy: 0.72, Test_data Accuracy: 0.46
Instrument: SZ300595, Train_data Accuracy: 0.73, Test_data Accuracy: 0.48
Instrument: SZ300601, Train_data Accuracy: 0.70, Test_data Accuracy: 0.63
Instrument: SZ300628, Train_data Accuracy: 0.70, Test_data Accuracy: 0.49
Instrument: SZ300661, Train_data Accuracy: 0.70, Test_data Accuracy: 0.52
Instrument: SZ300676, Train_data Accuracy: 0.66, Test_data Accuracy: 0.50
Instrument: SZ300677, Train_data Accuracy: 0.69, Test_data Accuracy: 0.46
Instrument: SZ300750, Train_data Accuracy: 0.68, Test_data Accuracy: 0.52
Instrument: SZ300751, Train_data Accuracy: 0.71, Test_data Accuracy: 0.43
Instrument: SZ300759, Train_data Accur

C:\Users\huangtuo\AppData\Local\Temp\ipykernel_23392\278371622.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Predicted_Signal'] = pd.concat(predicted_signals, keys=instruments).reorder_levels(['instrument', 'datetime'])


In [13]:
test.head()
test = test.rename(columns={'Predicted_Signal': 'rank'})
test.index = test.index.rename({'instrument':'code'})
test = test.reset_index('code')
test.to_csv("c:\\temp\\ranked_data_20240704.csv")

In [15]:
test.tail()

,code,open,high,low,close,volume,rank
datetime,,,,,,,
2024-05-08,SZ301269,1.061077,1.064147,1.038120,1.051200,1510169.000,-1
2024-05-09,SZ301269,1.050615,1.111474,1.050615,1.103733,3038486.250,1
2024-05-10,SZ301269,1.103751,1.107755,1.082530,1.086000,1202791.750,-1
2024-05-13,SZ301269,1.080130,1.103220,1.065049,1.083333,1934781.375,-1
2024-05-14,SZ301269,1.079593,1.099479,1.070651,1.086533,1349050.125,1


In [12]:
start_date = '2023-05-23'
end_date = '2024-05-15'

mask = (test.index.get_level_values('datetime') >= start_date) & (test.index.get_level_values('datetime') <= end_date)
result = test.loc[mask]

In [13]:
result.head()

open       high        low      close  \
instrument datetime                                                 
SH600000   2023-05-23  10.455962  10.525299  10.331157  10.331157   
           2023-05-24  10.303282  10.317149  10.109141  10.109141   
           2023-05-25  10.067789  10.164862  10.040054  10.137127   
           2023-05-26  10.137252  10.206590  10.054046  10.151119   
           2023-05-29  10.150387  10.344520  10.136520  10.275187   

                              volume  Predicted_Signal  
instrument datetime                                     
SH600000   2023-05-23  210665.843750                -1  
           2023-05-24  325560.375000                 1  
           2023-05-25  248127.109375                 1  
           2023-05-26  136381.078125                 1  
           2023-05-29  213635.500000                -1

In [14]:
result = result.rename(columns={'Predicted_Signal': 'rank'})
result.index = result.index.rename({'instrument':'code'})
result = result.reset_index('code')


In [15]:
test = test.rename(columns={'Predicted_Signal': 'rank'})

In [16]:
ranked_data_new = test

In [23]:
split = int(0.95 * len(benchmark))

In [28]:
result.head()

,code,open,high,low,close,volume,rank
datetime,,,,,,,
2023-05-23,SH600000,10.455962,10.525299,10.331157,10.331157,210665.843750,-1
2023-05-24,SH600000,10.303282,10.317149,10.109141,10.109141,325560.375000,1
2023-05-25,SH600000,10.067789,10.164862,10.040054,10.137127,248127.109375,1
2023-05-26,SH600000,10.137252,10.206590,10.054046,10.151119,136381.078125,1
2023-05-29,SH600000,10.150387,10.344520,10.136520,10.275187,213635.500000,-1


In [26]:
benchmark_new=benchmark[split:]

In [22]:
benchmark_old = ["SH000300"]
# data, benchmark = get_backtest_data(ranked_data, test_period[0], test_period[1], market, benchmark_old)
benchmark: pd.DataFrame = D.features(
    benchmark_old,
    fields=["$close"],
    start_time=test_period[0],
    end_time=test_period[1],
).reset_index(level=0, drop=True)
benchmark_ret: pd.Series = benchmark['$close'].pct_change()

In [29]:
import sys
import os
local_path = os.getcwd()
local_path = "C:/Users/huangtuo/Documents\\GitHub\\PairsTrading\\multi-fund\\"
sys.path.append(local_path+'\\Local_library\\')
from hugos_toolkit.BackTestTemplate import TopicStrategy,get_backtesting,AddSignalData
from hugos_toolkit.BackTestReport.tear import analysis_rets
from hugos_toolkit.BackTestTemplate import LowRankStrategy_new


In [30]:
test_period_1 = ("2023-05-23", "2024-05-15")
bt_result = get_backtesting(
    result,
    name="code",
    strategy=LowRankStrategy_new,
    mulit_add_data=True,
    feedsfunc=AddSignalData,
    strategy_params={"selnum": 5, "pre": 0.05, 'ascending': False, 'show_log': False},
    begin_dt=test_period_1[0],
    end_dt=test_period_1[1],
)

Strategy completed


In [31]:
trade_logger = bt_result.result[0].analyzers._trade_logger.get_analysis()
TradeListAnalyzer = bt_result.result[0].analyzers._TradeListAnalyzer.get_analysis()

OrderAnalyzer = bt_result.result[0].analyzers._OrderAnalyzer.get_analysis()

trader_df = pd.DataFrame(trade_logger)
orders_df = pd.DataFrame(OrderAnalyzer)

In [34]:
algorithm_returns: pd.Series = pd.Series(
    bt_result.result[0].analyzers._TimeReturn.get_analysis()
)
report = analysis_rets(algorithm_returns, bt_result.result, benchmark_new['$close'].pct_change(), use_widgets=True)

from plotly.offline import iplot
from plotly.offline import init_notebook_mode

init_notebook_mode()
for chart in report:
    iplot(chart)

ValueError: Neither `start` nor `end` can be NaT

In [36]:
bt_result.result